# 🚀 Modelos Anidados en Pydantic v2
Este módulo muestra cómo definir modelos anidados en Pydantic v2. Pydantic es una librería de Python que permite definir modelos de datos y validarlos. En la versión 2 de Pydantic se introdujo la posibilidad de definir modelos anidados, es decir, modelos que contienen otros modelos. En este módulo se muestra cómo definir y utilizar modelos anidados en Pydantic v2.

In [ ]:
# Si pydantic no está instalado ejecuta la proxima línea quitándole la marca de comentario
# pip install pydantic

Pydantic v2 (lanzado en 2023) permite crear modelos anidados, lo que significa que puedes definir un modelo que contenga instancias de otros modelos como atributos. 
- Esto es extremadamente útil para representar datos con estructuras jerárquicas complejas.

Características importantes de los modelos anidados en Pydantic
1. Validación en cascada: Las validaciones se aplican a todos los niveles, desde el modelo raíz hasta el más anidado.
2. Serialización completa: Puedes convertir toda la estructura anidada a JSON u otros formatos.
3. Rendimiento mejorado: Pydantic v2 está implementado en Rust (mediante PyO3), lo que lo hace mucho más rápido que la v1.
4. Referencias circulares: Puedes crear modelos con referencias circulares usando strings para los tipos y model_rebuild().
5. Actualización desde diccionarios: Puedes actualizar estructuras anidadas pasando diccionarios con la estructura correspondiente.

## 📌 Modelo anidado por composición
En este ejemplo, definimos dos modelos anidados `Address` y `Person` utilizando la composición. 
- La clase `Address` define un modelo de dirección con los atributos `street`, `city` y `zip_code`. 
- La clase `Person` define un modelo de persona con los atributos `name`, `age` y `address` que es una instancia del modelo `Address`.
- El modelo `Person` contiene una instancia del modelo `Address` como atributo.

In [10]:
from pydantic import BaseModel

class Address(BaseModel):
    street: str
    city: str
    zip_code: str
    country: str

class Person(BaseModel):
    name: str
    email: str
    address: Address

# Uso del modelo
address = Address(
    street="Calle Principal 123",
    city="Mendoza",
    zip_code="5509",
    country="Argentina"
)

user = Person(
    name="Juan Pérez",
    email="juan@ejemplo.com",
    address=address
)

# También puedes crear directamente con un diccionario
user2 = Person(
    name="María López",
    email="maria@ejemplo.com",
    address={
        "street": "Avenida Central 456",
        "city": "Barcelona", 
        "zip_code": "08001",
        "country": "España"
    }
)

print(user.model_dump_json(indent=2))
print("Acceso a campos anidados:")
print(f"Ciudad: {user.address.city}")
print(f"\n{user2.model_dump_json()}")
print(f"Nombre de pila: {user2.name.split()[0]}")


{
  "name": "Juan Pérez",
  "email": "juan@ejemplo.com",
  "address": {
    "street": "Calle Principal 123",
    "city": "Mendoza",
    "zip_code": "5509",
    "country": "Argentina"
  }
}
Acceso a campos anidados:
Ciudad: Mendoza

{"name":"María López","email":"maria@ejemplo.com","address":{"street":"Avenida Central 456","city":"Barcelona","zip_code":"08001","country":"España"}}
Nombre de pila: María


## 📌 Lista de modelos anidados

En este caso se preve estructuras de datos anidadas, como una lista de direcciones en el modelo `Person`, donde cada dirección es una instancia del modelo `Address`.
- Esto serviría para representar una persona con múltiples direcciones.

Pero veamos este concepto con otro ejemplo.
- Supongamos que tenemos un modelo `Post` que contiene una lista de tags, donde cada tag es una instancia del modelo `Tag` .

In [11]:
from typing import List
from pydantic import BaseModel

class Tag(BaseModel):
    name: str
    value: str

class Post(BaseModel):
    title: str
    content: str
    tags: List[Tag]  # Lista de modelos anidados

# Creación de instancias
post = Post(
    title="Guía de Pydantic",
    content="Pydantic es una librería para validación de datos...",
    tags=[
        {"name": "python", "value": "programming"},
        {"name": "pydantic", "value": "validation"},
        {"name": "tutorial", "value": "guide"}
    ]
)

# Acceso a los elementos de la lista
print(f"Título: {post.title}")
print("Etiquetas:")
for tag in post.tags:
    print(f"  - {tag.name}: {tag.value}")

Título: Guía de Pydantic
Etiquetas:
  - python: programming
  - pydantic: validation
  - tutorial: guide


## 📌 Modelos anidados recursivos
Recursivo significa que un modelo puede contener instancias de sí mismo.
- Un ejemplo podría ser definir un modelo `Category` que contiene una lista de subcategorías, donde cada subcategoría es una instancia del modelo `Category`.
 
En este caso, definimos un modelo `Comment` que contiene una lista de respuestas, donde cada respuesta es una instancia del modelo `Comment`.

In [13]:

from typing import List, Optional
from pydantic import BaseModel

class Comment(BaseModel):
    content: str
    author: str
    replies: List["Comment"] = []  # Modelo recursivo
    
# Resolución de tipos hacia adelante
Comment.model_rebuild()

# Creación de una estructura de comentarios anidados
comments = Comment(
    content="Gran artículo, gracias por compartir!",
    author="usuario1",
    replies=[
        Comment(
            content="Estoy de acuerdo, muy útil",
            author="usuario2",
            replies=[]
        ),
        Comment(
            content="¿Podrías explicar más sobre la sección 3?",
            author="usuario3",
            replies=[
                Comment(
                    content="La sección 3 trata sobre...",
                    author="usuario1",
                    replies=[]
                )
            ]
        )
    ]
)

# Acceso a la estructura anidada
print(f"Comentario principal: {comments.content} (por {comments.author})")
print(f"Número de respuestas: {len(comments.replies)}")
print(f"Primer respuesta: {comments.replies[0].content}")
print(f"Respuesta a respuesta: {comments.replies[1].replies[0].content}")

Comentario principal: Gran artículo, gracias por compartir! (por usuario1)
Número de respuestas: 2
Primer respuesta: Estoy de acuerdo, muy útil
Respuesta a respuesta: La sección 3 trata sobre...


## 📌 Modelos anidados con discriminadores

In [14]:
from typing import Literal, Union
from pydantic import BaseModel, Field

class Circle(BaseModel):
    type: Literal["circle"]
    radius: float

class Rectangle(BaseModel):
    type: Literal["rectangle"]
    width: float
    height: float

class Triangle(BaseModel):
    type: Literal["triangle"]
    base: float
    height: float

class Drawing(BaseModel):
    name: str
    shapes: list[Union[Circle, Rectangle, Triangle]]

# Crear un dibujo con diferentes formas
drawing = Drawing(
    name="Formas básicas",
    shapes=[
        {"type": "circle", "radius": 5.0},
        {"type": "rectangle", "width": 10.0, "height": 20.0},
        {"type": "triangle", "base": 15.0, "height": 8.0}
    ]
)

# Acceso y procesamiento de las formas
print(f"Dibujo: {drawing.name}")
print("Formas:")
for shape in drawing.shapes:
    if isinstance(shape, Circle):
        area = 3.14159 * shape.radius ** 2
        print(f"  - Círculo con radio {shape.radius}, área: {area:.2f}")
    elif isinstance(shape, Rectangle):
        area = shape.width * shape.height
        print(f"  - Rectángulo {shape.width}x{shape.height}, área: {area:.2f}")
    elif isinstance(shape, Triangle):
        area = 0.5 * shape.base * shape.height
        print(f"  - Triángulo con base {shape.base} y altura {shape.height}, área: {area:.2f}")

Dibujo: Formas básicas
Formas:
  - Círculo con radio 5.0, área: 78.54
  - Rectángulo 10.0x20.0, área: 200.00
  - Triángulo con base 15.0 y altura 8.0, área: 60.00


## 📌 Vaidación en modelos anidados

In [16]:
from typing import List
from pydantic import BaseModel, Field, field_validator

class Location(BaseModel):
    latitude: float = Field(ge=-90, le=90)
    longitude: float = Field(ge=-180, le=180)
    
    @field_validator('latitude')
    @classmethod
    def check_lat_precision(cls, v: float) -> float:
        if abs(v) == 90 and v % 1 != 0:
            raise ValueError("Latitudes of exactly +/-90 cannot have decimal places")
        return v

class Venue(BaseModel):
    name: str
    location: Location
    capacity: int = Field(gt=0)
    
class Event(BaseModel):
    name: str
    description: str = ""
    venues: List[Venue]
    
    @field_validator('venues')
    @classmethod
    def at_least_one_venue(cls, venues: List[Venue]) -> List[Venue]:
        if not venues:
            raise ValueError("Event must have at least one venue")
        return venues

# Uso de los modelos con validación
try:
    event = Event(
        name="Conferencia de Python",
        description="Una gran conferencia sobre Python y sus bibliotecas",
        venues=[
            {
                "name": "Teatro Principal",
                "location": {"latitude": 40.416775, "longitude": -3.703790},
                "capacity": 500
            },
            {
                "name": "Sala de Talleres", 
                "location": {"latitude": 40.415, "longitude": -3.705},
                "capacity": 100
            }
        ]
    )
    print("Evento válido creado:")
    print(f"Nombre: {event.name}")
    print(f"Número de sedes: {len(event.venues)}")
    
    # Esto fallará por la validación
    invalid_event = Event(
        name="Evento inválido",
        venues=[
            {
                "name": "Sede con coordenadas inválidas",
                "location": {"latitude": 91, "longitude": 0},  # Latitud inválida
                "capacity": 200
            }
        ]
    )
    
except ValueError as e:
    print(f"Error de validación: {e}")

Evento válido creado:
Nombre: Conferencia de Python
Número de sedes: 2
Error de validación: 1 validation error for Event
venues.0.location.latitude
  Input should be less than or equal to 90 [type=less_than_equal, input_value=91, input_type=int]
    For further information visit https://errors.pydantic.dev/2.10/v/less_than_equal


Autor: Daniel Christello
______________________________________________________